# CPPI Strategy Development
This notebook develops and tests the CPPI core logic for the trading bot.


In [1]:
import sys, os
sys.path.append(os.path.abspath(".."))  # This allows you to import from the parent directory
from strategies.cppi import CPPI
from strategies.asset_selector import SimpleAssetSelector
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [14]:
# CPPI parameters
initial_capital = 100000
floor_pct = 0.8
multiplier = 3
cost_per_dollar = 0.0005   # 5 basis points per dollar traded

# Date range
start_date = '2005-01-01'
end_date   = '2023-01-01'

# Initialize CPPI engine and asset selector
cppi = CPPI(floor_pct, multiplier, initial_capital)
selector = SimpleAssetSelector()


In [15]:
# Cell 4: Data Acquisition using yfinance.download

import pandas as pd
import yfinance as yf

# Parameters
start_date = '2005-01-01'
end_date   = '2023-01-02'         # add one extra day to include Jan 1 data
tickers    = ['SPY', 'TLT']       # your risky and safe assets

# Download OHLC data for both tickers
data = yf.download(
    tickers,
    start=start_date,
    end=end_date,
    interval='1d',
    group_by='ticker',
    auto_adjust=True,
    threads=True
)

# Extract Close-price Series
risky_prices = data['SPY']['Close']
safe_prices  = data['TLT']['Close']

# Align and compute returns
prices = pd.concat([risky_prices, safe_prices], axis=1, join='inner')
prices.columns = ['risky', 'safe']
returns = prices.pct_change().fillna(0)

# Sanity check
display(returns.head())


[*********************100%***********************]  2 of 2 completed


,risky,safe
Date,,
2005-01-03,0.000000,0.000000
2005-01-04,-0.012219,-0.010481
2005-01-05,-0.006901,0.005354
2005-01-06,0.005084,0.000679
2005-01-07,-0.001433,0.002264


In [16]:
# Simulation loop
portfolio_vals = []
previous_risky_alloc = 0

for date, row in returns.iterrows():
    # Rebalance with cost
    new_risky_alloc, cost = cppi.rebalance_with_cost(
        risky_return=row['risky'],
        safe_return =row['safe'],
        previous_risky_alloc=previous_risky_alloc,
        cost_per_dollar=cost_per_dollar
    )
    previous_risky_alloc = new_risky_alloc
    portfolio_vals.append({
        'date': date,
        'value': cppi.portfolio_value,
        'cost': cost
    })

# Build DataFrame
cppi_df = pd.DataFrame(portfolio_vals).set_index('date')


In [17]:
# Cell: Interactive CPPI vs. Buy & Hold with Sliders (no external CSV)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact_manual, FloatSlider
from strategies.cppi import CPPI

# Use the existing `returns` DataFrame in this notebook
# It should have columns ['risky', 'safe'] and a DatetimeIndex
# Ensure you've run the data-acquisition cell so that `returns` is defined.

def plot_cppi(floor_pct: float, multiplier: float):
    initial_capital = 100_000
    cppi = CPPI(floor_pct=floor_pct, multiplier=multiplier, initial_capital=initial_capital)
    dates = returns.index

    # Run CPPI simulation
    port_vals = []
    for rr, sr in zip(returns['risky'], returns['safe']):
        cppi.rebalance(rr, sr)
        port_vals.append(cppi.portfolio_value)
    cppi_series = pd.Series(port_vals, index=dates)

    # Buy & Hold benchmark
    bh_series = (1 + returns['risky']).cumprod() * initial_capital

    # Plot
    plt.figure(figsize=(12, 6))
    plt.plot(dates, cppi_series, label=f'CPPI (floor={floor_pct:.2f}, m={multiplier:.1f})', linewidth=2)
    plt.plot(dates, bh_series, label='Buy & Hold', linestyle='--')
    plt.title('CPPI vs. Buy & Hold Performance')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value ($)')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()

# Use a manual “Run” button to avoid hanging during slider moves
interact_manual(
    plot_cppi,
    floor_pct=FloatSlider(value=0.8, min=0.1, max=0.9, step=0.05, description='Floor %'),
    multiplier=FloatSlider(value=3.0, min=0.5, max=5.0, step=0.5, description='Multiplier')
);


interactive(children=(FloatSlider(value=0.8, description='Floor %', max=0.9, min=0.1, step=0.05), FloatSlider(…

In [6]:
from ipywidgets import interact, FloatSlider
print("Widgets are now available!")


Widgets are now available!
